 MACHINE LEARNING ASSIGNMENT


 Hafsa Naz


23f-AI-56
 (A2)

Task: Classification on Bank Marketing Dataset

IMPORT LIBRARIES

In [1]:
# import all required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


IMPORT DATASET

In [2]:
# Load the datset
df=pd.read_csv('/content/bank-full.csv',sep=';')
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


EDA

In [3]:
df.shape   # shape of datset

(45211, 17)

In [4]:
df.columns  # columns name in the datset

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [7]:
df.dtypes  # type of datset

,0
age,int64
job,object
marital,object
education,object
default,object
balance,int64
housing,object
loan,object
contact,object
day,int64


In [8]:
df.isnull().sum()  # check the null values based on numerical datset

,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0
day,0


In [9]:
df['y'].value_counts(normalize=True)  # check the predicted column proportion


,proportion
y,
no,0.883015
yes,0.116985


DATA PREPROCESSING

1. Handle missing values

In [10]:
(df == 'unknown').sum() # check the categorical null values


,0
age,0
job,288
marital,0
education,1857
default,0
balance,0
housing,0
loan,0
contact,13020
day,0


The job column has relatively few missing values, so these can be filled with the most frequent value (mode). The contact and education columns have around 30% missing values, so we can create a new category called "unknown" to let the model learn from these missing entries instead of discarding rows. The poutcome column has a very high percentage of missing values, so it is best to drop this column entirely.

In [11]:
df['job'].fillna(df['job'].mode()[0], inplace=True) # fill the job column with its most frequent value

/tmp/ipython-input-26646152.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['job'].fillna(df['job'].mode()[0], inplace=True) # fill the job column with its most frequent value


In [12]:
df['contact'].fillna('unknown', inplace=True)  # fill null columns with unknown
df['education'].fillna('unknown', inplace=True)


/tmp/ipython-input-1080391970.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['contact'].fillna('unknown', inplace=True)  # fill null columns with unknown
/tmp/ipython-input-1080391970.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

In [13]:
df.drop('poutcome', axis=1, inplace=True)

2. Normalize and standarize features

In [14]:
cat_cols = ['job','marital','education','default','housing','loan','contact','month','y']
le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col])  # encode the categorical columns


In [15]:
num_cols = ['age','balance','day','duration','campaign','pdays','previous']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols]) # standarize numerical columns


MODEL DEVELOPMENT

In [16]:
X = df.drop('y', axis=1)    #spilt the data set in train and test
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Decision tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)



DecisionTreeClassifier(random_state=42)

In [18]:
#Naive bayes
nb = GaussianNB()
nb.fit(X_train, y_train)


GaussianNB()

In [19]:
#Support vector machine
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)


SVC(probability=True, random_state=42)

MODELS EVALUATION

In [20]:
models = {'Decision Tree': dt, 'Naive Bayes': nb, 'SVM': svm}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-Score:", f1_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("\n")


Decision Tree
Accuracy: 0.873382726971138
Precision: 0.47527472527472525
Recall: 0.47571035747021084
F1-Score: 0.4754924415941365
[[7379  573]
 [ 572  519]]


Naive Bayes
Accuracy: 0.8457370341700763
Precision: 0.3869047619047619
Recall: 0.4766269477543538
F1-Score: 0.4271047227926078
[[7128  824]
 [ 571  520]]


SVM
Accuracy: 0.8849939179475838
Precision: 0.650887573964497
Recall: 0.1008249312557287
F1-Score: 0.1746031746031746
[[7893   59]
 [ 981  110]]




ENSEMBLE (BAGGING/BOOSTING)

In [21]:
#Bagging with decision tree
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
bagging.fit(X_train, y_train)
y_pred_bag = bagging.predict(X_test)


In [22]:
#Boosting with adaboost
boost = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
boost.fit(X_train, y_train)
y_pred_boost = boost.predict(X_test)


EVALUTE ENSEMBLE

In [23]:
ensembles = {'Bagging': y_pred_bag, 'Boosting': y_pred_boost}

for name, y_pred in ensembles.items():
    print(f"{name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-Score:", f1_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("\n")


Bagging
Accuracy: 0.9002543403737697
Precision: 0.6159509202453988
Recall: 0.46012832263978004
F1-Score: 0.5267576075550892
[[7639  313]
 [ 589  502]]


Boosting
Accuracy: 0.8717239854030742
Precision: 0.4684354986276304
Recall: 0.4692942254812099
F1-Score: 0.46886446886446886
[[7371  581]
 [ 579  512]]




Impact of overfitting and how pruning or regularization
mitigated

Overfitting occurs when a model learns noise and specific patterns from the training data, resulting in poor generalization to unseen data. In this study, Decision Trees showed a tendency to overfit due to their high variance nature. Pruning techniques such as limiting tree depth and minimum samples per split reduced model complexity and improved generalization. Support Vector Machines mitigated overfitting through regularization using the C parameter. Additionally, ensemble methods like Bagging reduced variance by averaging multiple models, leading to improved performance on noisy data.